In [1]:
from Levenshtein import distance
import numpy as np
import pandas as pd
import os
import unicodedata
import re

In [2]:
def get_levenshtein(x,y):
    # ordering the inputs by length so that a<=b
    a=max(len(x), len(y))
    b=min(len(x), len(y))
    if a==0: # one of the input is empty
        return 1.
    if a > b+500: # if there are more than 500 more chars then return 100%
        return 1.
    # return the levenshtein distance normalized by the mean length
    mean_length = (a + b)/2.
    return 2 * distance(x,y)/mean_length

def normalize_text(x, to_upper = True):
    ans = unicodedata.normalize("NFKD", x.replace(";"," "))
    if(to_upper):
        return ans.upper()
    return ans
    

In [3]:
def keep_digits(x):
    return ''.join(filter(lambda x: x.isdigit(), x))

In [4]:
class Amendement:
    def __init__(self, txt, start_signal, expose_signal):
        
        self.full_txt = txt
        self.full_txt_normalized = [normalize_text(t) for t in txt]
        self.start_signal = normalize_text(start_signal)
        self.expose_signal = normalize_text(expose_signal)
        #self.start_offset = start_offset
        
        self.numero_amendement = self.get_amendement_nb()
        
        self.numero_article = self.get_amendement_article()
        
        self.txt_normalized, self.txt = self.get_amendement_text()
        
        
    def get_amendement_nb(self):
        x = self.full_txt_normalized
        for i in range(0, len(x)):
            if x[i][0:3]=="N° ":
                return x[i].replace("N° ", "")
        return "numéro amendement introuvable 0"
    
    def get_amendement_article(self):
        has_started=False
        x = self.full_txt_normalized
        for i in range(0, len(x)):
            if x[i]==self.start_signal:
                has_started = True

            if(has_started) and ('ARTICLE' in x[i] or 'TITRE' in x[i]):
                self.article_line = i
                return re.sub(".*ARTICLE ","",x[i])
        return ""
    
    def get_amendement_text(self):
        x = self.full_txt_normalized
        y = self.full_txt
        start,end=0,0
        
        expose_length = len(self.expose_signal)
        
        start = self.article_line+1
        
        for i in range(self.article_line+1, len(x)):

            if i<len(x)-3 and len(x[i])==0 and len(x[i+1])==0 and len(x[i+2])==0:
                end=i

            if len(x[i])<18 and x[i][0:expose_length]==self.expose_signal:
                end=i
                break
        candidate_normalized = x[start:end]
        candidate_raw = y[start:end]
        res_normalized, res_raw =[], []
        for i in range(0, len(candidate_normalized)):
            if normalize_text("Cet amendement est en cours") in candidate_normalized[i]:
                continue
            elif len(candidate_normalized[i]) < 2:
                continue
            else:
                res_normalized.append(candidate_normalized[i])
                res_raw.append(candidate_raw[i])
        return " ".join(res_normalized).replace("  ", " "), \
               " ".join(res_raw).replace("  ", " ")


In [5]:
class Liasse:
    
    def guess_projet_loi(self):
        df_txt = pd.DataFrame({"line":self.all_txt, "one":1})
        self.df_txt = df_txt
        
        df_txt_count = pd.DataFrame(df_txt.groupby("line")["one"].sum()).reset_index().sort_values(by='one', ascending=False)
        df_txt_count["len"] = df_txt_count.line.apply(lambda x:len(x))
        self.df_txt_count = df_txt_count
        #print(df_txt_count.head(10))
        
        
        for i, row in self.df_txt_count.head(20).iterrows():
            if row.len >= 45:
                self.projet_loi = row.line.strip()
                print("Projet de loi détecté : " + self.projet_loi)
                break
                
        self.start_offset_before_loi = df_txt[df_txt.line==self.projet_loi].head(1).index.values[0]
        print("Offset avant loi détecté : {}".format(self.start_offset_before_loi))
        
        
    def guess_expose_signal(self):
        for i, row in self.df_txt_count.head(20).iterrows():
            if normalize_text(row.line)[0:5] == "OBJET" or \
               normalize_text(row.line)[0:5] == "EXPOS":
                self.amendement_expose_signal = normalize_text(row.line)
                print("Exposé détecté : " + row.line)
                break
                
    def guess_amendement_start(self):
        for i, row in self.df_txt_count.head(20).iterrows():
            if normalize_text(row.line)[0:4] == "----":
                break
            if normalize_text(row.line) == normalize_text("présenté par"):
                break

                
        self.amendement_start_signal = normalize_text(row.line)
        print("Debut d'amendement : " + row.line)
        
        
    def __init__(self, folder): 
            self.folder = folder
            #self.projet_loi = normalize_text(projet_loi)
            #start_offset_before_loi = start_offset
            self.nb_amendements = 0
            
            
            #row=0
            all_txt, all_txt_normalized = [], []
            print("Lecture des fichiers dans {} : ".format(folder))
            for f in os.listdir(folder):
                
                current_file= open(folder+"/"+f, encoding="iso-8859-1")
                if f[0:1]==".":
                    continue
                    
                print("{}".format(f), end=',')
                for line in current_file:
                    all_txt.append(normalize_text(line.strip(), False))
                    all_txt_normalized.append(normalize_text(line.strip()))

                current_file.close()
            print("")
            print("")

            self.all_txt = all_txt
            
            self.guess_projet_loi()
            self.guess_expose_signal()
            self.guess_amendement_start()
            
            starts=[]
            for i, row in self.df_txt.iterrows():
                if normalize_text(row["line"].strip()[0:len(self.projet_loi)]) == normalize_text(self.projet_loi):
                        starts.append(i-self.start_offset_before_loi)
            
            
            begins, ends, amendements = [],[],[]
            amendements_nb, amendements_txt, amendements_txt_normalized, amendements_art = [], [], [], []
            
            for i in range(0, len(starts)):
                if i<len(starts)-1:
                    begin=starts[i]
                    end=starts[i+1]
                else:
                    begin=starts[-1]
                    end=len(all_txt)

                current_amendement = Amendement(all_txt[begin:end], self.amendement_start_signal, \
                                                self.amendement_expose_signal)
                
                begins.append(begin)
                ends.append(end)
                amendements.append(current_amendement)
                
                amendements_nb.append(current_amendement.numero_amendement)
                amendements_txt.append(current_amendement.txt)
                amendements_txt_normalized.append(current_amendement.txt_normalized)
                amendements_art.append(current_amendement.numero_article)
                
            self.begins = begins    
            self.ends = ends
            self.amendements = amendements

            self.df_amendements = pd.DataFrame({'nb':amendements_nb, \
                                          'txt_brut':amendements_txt, \
                                          'txt':amendements_txt_normalized, \
                                          'article':amendements_art, 'begin':begins, 'end':ends})
            print("Il y a {} amendements dans la liasse {}".format(len(self.df_amendements), self.folder))
        
        
            

In [12]:
liasse_AN = Liasse("AN/")
liasse_AN.df_amendements.head()

Lecture des fichiers dans AN/ : 
2 _ art 11- apräs 11.txt,6_ apräs 13 ter- apräs 14.txt,9_ art 15.txt,5_ art 12 bis -apräs 13 ter.txt,8_ 14 bis -apräs 14 undecies.txt,3_ 11 bis apräs 11 quindecies.txt,11_art 15 bis Ö avant 16.txt,10_ apräs 15.txt,1_  avant 11 et 11 liasse 2.txt,4- art 11 sexdecies -apräs 12.txt,12- art 16 Ö la fin.txt,

Projet de loi détecté : EQUILIBRE DANS LE SECTEUR AGRICOLE ET ALIMENTAIRE - (N° 902)
Offset avant loi détecté : 5
Exposé détecté : EXPOSÉ SOMMAIRE
Debut d'amendement : ----------
Il y a 1379 amendements dans la liasse AN/


,nb,txt_brut,txt,article,begin,end
0,341,"APRÈS L'ARTICLE 11, insérer l'article suivan...","APRÈS L'ARTICLE 11, INSÉRER L'ARTICLE SUIVAN...",ADDITIONNEL,0,27
1,256,"APRÈS L'ARTICLE 11, insérer l'article suivan...","APRÈS L'ARTICLE 11, INSÉRER L'ARTICLE SUIVAN...",ADDITIONNEL,27,56
2,7,"APRÈS L'ARTICLE 11, insérer l'article suivan...","APRÈS L'ARTICLE 11, INSÉRER L'ARTICLE SUIVAN...",ADDITIONNEL,56,86
3,8,"APRÈS L'ARTICLE 11, insérer l'article suivan...","APRÈS L'ARTICLE 11, INSÉRER L'ARTICLE SUIVAN...",ADDITIONNEL,86,119
4,9,"APRÈS L'ARTICLE 11, insérer l'article suivan...","APRÈS L'ARTICLE 11, INSÉRER L'ARTICLE SUIVAN...",ADDITIONNEL,119,150


In [54]:
df_avis_AN = pd.read_excel("dérouleur AN séance publique.xlsx", skiprows=[0], sheet_name="dérouleur retraité")
df_avis_AN["N°"] = df_avis_AN["N°"].astype(str)
df_avis_AN_small = df_avis_AN[["N°", "SAJ"]]
df_avis_AN_small.columns = ["N°", "avis"]

liasse_AN_avec_avis = pd.merge(liasse_AN.df_amendements, df_avis_AN_small, left_on="nb", right_on="N°")
liasse_AN_avec_avis.head()

#print(len(liasse_AN_avec_avis))
#print(len(df_avis_AN_small))

#liasse_AN_avec_avis[liasse_AN_avec_avis.SAJ==0]

,nb,txt_brut,txt,article,begin,end,N°,avis
0,341,"APRÈS L'ARTICLE 11, insérer l'article suivan...","APRÈS L'ARTICLE 11, INSÉRER L'ARTICLE SUIVAN...",ADDITIONNEL,0,27,341,Opportunité
1,256,"APRÈS L'ARTICLE 11, insérer l'article suivan...","APRÈS L'ARTICLE 11, INSÉRER L'ARTICLE SUIVAN...",ADDITIONNEL,27,56,256,cf. DGCCRF
2,7,"APRÈS L'ARTICLE 11, insérer l'article suivan...","APRÈS L'ARTICLE 11, INSÉRER L'ARTICLE SUIVAN...",ADDITIONNEL,56,86,7,Défavorable (cf. amendement CE260) : prématuré...
3,8,"APRÈS L'ARTICLE 11, insérer l'article suivan...","APRÈS L'ARTICLE 11, INSÉRER L'ARTICLE SUIVAN...",ADDITIONNEL,86,119,8,Opportunité (cf. amendement CE261)
4,9,"APRÈS L'ARTICLE 11, insérer l'article suivan...","APRÈS L'ARTICLE 11, INSÉRER L'ARTICLE SUIVAN...",ADDITIONNEL,119,150,9,Défavorable (cf. amendement n° 331) :\n1 - Pro...


In [36]:
liasse_senat = Liasse("senat/")
liasse_senat.df_amendements["nb_figures"] = liasse_senat.df_amendements["nb"].apply(lambda x:x.replace('COM-',''))
liasse_senat.df_amendements.head()

Lecture des fichiers dans senat/ : 
liasse sénat 1- 362 COMECO.txt,

Projet de loi détecté : Relations commerciales dans le secteur agricole et alimentaire
Offset avant loi détecté : 4
Exposé détecté : Objet
Debut d'amendement : présenté par
Il y a 259 amendements dans la liasse senat/


,nb,txt_brut,txt,article,begin,end,nb_figures
0,COM-1,Après l'article 15 bis (nouveau) Insérer un ...,APRÈS L'ARTICLE 15 BIS (NOUVEAU) INSÉRER UN ...,15,0,36,1
1,COM-2,Après l'article 15 bis (nouveau) Insérer un ...,APRÈS L'ARTICLE 15 BIS (NOUVEAU) INSÉRER UN ...,15 BIS (NOUVEAU),36,66,2
2,COM-3,Après l'article 11 sexies (nouveau) Insérer ...,APRÈS L'ARTICLE 11 SEXIES (NOUVEAU) INSÉRER ...,11 SEXIES (NOUVEAU),66,120,3
3,COM-4,Après l'article 14 quater (nouveau) Insérer ...,APRÈS L'ARTICLE 14 QUATER (NOUVEAU) INSÉRER ...,14 QUATER (NOUVEAU),120,146,4
4,COM-5,Après l'article 14 quater (nouveau) Insérer ...,APRÈS L'ARTICLE 14 QUATER (NOUVEAU) INSÉRER ...,14 QUATER (NOUVEAU),146,180,5


In [55]:
df_avis_senat = pd.read_excel("Dérouleur ComECO  Senat.xlsx", skiprows=[0])
df_avis_senat["N°"] = df_avis_senat["N°"].astype(str)
df_avis_senat_small = df_avis_senat[["N°", "Avis SAJ"]]
df_avis_senat_small.columns = ["N°", "avis"]
df_avis_senat_small.head()

print(len(df_avis_senat_small))

liasse_senat_avec_avis = pd.merge(liasse_senat.df_amendements, df_avis_senat_small, left_on="nb_figures", right_on="N°")

liasse_senat_avec_avis.head()

404


,nb,txt_brut,txt,article,begin,end,nb_figures,N°,avis
0,COM-1,Après l'article 15 bis (nouveau) Insérer un ...,APRÈS L'ARTICLE 15 BIS (NOUVEAU) INSÉRER UN ...,15,0,36,1,1,Cavalier
1,COM-2,Après l'article 15 bis (nouveau) Insérer un ...,APRÈS L'ARTICLE 15 BIS (NOUVEAU) INSÉRER UN ...,15 BIS (NOUVEAU),36,66,2,2,Cavalier
2,COM-3,Après l'article 11 sexies (nouveau) Insérer ...,APRÈS L'ARTICLE 11 SEXIES (NOUVEAU) INSÉRER ...,11 SEXIES (NOUVEAU),66,120,3,3,"Défavorable \nMal rédigé, risque de contrariét..."
3,COM-4,Après l'article 14 quater (nouveau) Insérer ...,APRÈS L'ARTICLE 14 QUATER (NOUVEAU) INSÉRER ...,14 QUATER (NOUVEAU),120,146,4,4,x
4,COM-5,Après l'article 14 quater (nouveau) Insérer ...,APRÈS L'ARTICLE 14 QUATER (NOUVEAU) INSÉRER ...,14 QUATER (NOUVEAU),146,180,5,5,Opportunité


In [56]:
def compute_match(df_amendements_1, df_amendements_2, version1, version2, test_fast=False):
    
    threshold = 0.01

    corresp={}

    for i1, row1 in df_amendements_1.iterrows():
        
        if test_fast and (i1 > 30):
            break

        if i1 % 20 == 0:
            print( "{} %".format(int(i1 * 100 / len(df_amendements_1))), end="  ")
        for i2,row2 in df_amendements_2.iterrows():
            l = get_levenshtein(row1.txt,row2.txt)
            current_similarity = 1-l

            if(current_similarity > threshold):

                update = True
                if row2.nb in corresp:
                    previous_similarity = corresp[row2.nb][3]
                    if previous_similarity >= current_similarity:
                        update = False
                        
                if row1.txt =="SUPPRIMER CET ARTICLE." and \
                keep_digits(row1.article) != keep_digits(row2.article):
                    update = False

                if update:
                    corresp[row2.nb] = (row1.nb, row1.txt_brut, row2.txt_brut, \
                                        current_similarity, row1.article, row2.article, row1.avis)
                    
    amendement_id2, amendement_id1, amendement_txt2, amendement_txt1, similarity, article1, article2, avis1 \
    = [], [], [], [], [], [], [], []

    for id2 in corresp:
        id1, txt1, txt2, simi, art1, art2, avi1 = corresp[id2] 
        amendement_id2.append(str(id2))
        amendement_id1.append(str(id1))
        amendement_txt2.append(txt2)
        article1.append(art1)
        article2.append(art2)
        avis1.append(avi1)
        amendement_txt1.append(txt1)
        similarity.append("{0:.0%}".format(simi))

    df_amendements_matched = pd.DataFrame( { \
                                         "N° amendement "+version1:amendement_id1, \
                                         "N° amendement "+version2:amendement_id2, \
                                         "Amendement "+version1:amendement_txt1, \
                                         "Amendement "+version2:amendement_txt2, \
                                         "N° article de la loi ({})".format(version1):article1, \
                                         "N° article de la loi ({})".format(version2):article2, \
                                         "Similarité":similarity, \
                                           "Avis SAJ "+version1:avis1})
    print("Il y a {} amendements communs".format(len(df_amendements_matched)))
    
    df_amendements_matched["id2"] = df_amendements_matched["N° amendement "+version2]\
    .apply(lambda x:keep_digits(x))\
    .astype(int)
    df_amendements_matched.sort_values(by="id2", inplace = True, ascending=True)
    del df_amendements_matched["id2"]
    
    
    nb_id2_matched = len(df_amendements_matched["N° amendement "+version2].unique())
    nb_id1_matched = len(df_amendements_matched["N° amendement "+version1].unique())
    nb_id2 = len(df_amendements_2)
    nb_id1 = len(df_amendements_1)
    print("""Sur les {} amendements de la nouvelle liasse, {} ont été retrouvés dans la première liasse, \
    soit {:.0%}.
    Sur les {} amendements de l'ancienne liasse, {} ont été ré-utilisés dans la seconde liasse, \
    soit {:.0%}.
          """\
         .format(nb_id2, nb_id2_matched, nb_id2_matched/nb_id2, nb_id1, nb_id1_matched, nb_id1_matched/nb_id1))
    
    return df_amendements_matched

In [9]:
def match_amendement(df_amendements_2, df_amendements_matched, version1, version2):
    
    df_results = pd.merge(df_amendements_2, df_amendements_matched, left_on = "nb", right_on="N° amendement "+version2)

    df_results["N° amendement "+version2] = df_results["nb"]
    df_results["Amendement "+version2] = df_results["txt_brut"]

    del df_results["begin"], df_results["end"], df_results["nb"],  df_results["txt"],\
    df_results["txt_brut"], df_results["article"]

    df_results["N° amendement "+version1].fillna("non trouvé", inplace = True)
    df_results["Amendement "+version1].fillna("non trouvé", inplace = True)
    df_results["Similarité"].fillna("0%", inplace = True)
    
    df_results.to_excel("matching_{}_{}.xlsx".format(version1, version2), index=False)
    
    return df_results

mot="CUNICOLE"

df_amendements_1["cont"] = df_amendements_1["txt"].apply(lambda x:mot in x)

df_amendements_2["cont"] = df_amendements_2["txt"].apply(lambda x:mot in x)

df_amendements_1[df_amendements_1.cont==1]
df_amendements_2[df_amendements_2.cont==1]
del df_amendements_1["cont"]
del df_amendements_2["cont"]

In [ ]:
df_amendements_matched = compute_match(liasse_AN_avec_avis, liasse_senat_avec_avis, \
                                       "AN séance publiqe", "senat COM ECO", False)
df_amendements_matched.head(20)

0 %  1 %  2 %  4 %  5 %  7 %  8 %  10 %  11 %  

In [11]:
df_results = match_amendement(liasse_senat.df_amendements, df_amendements_matched, "AN", "senat")
df_results.head()

,N° amendement AN,N° amendement senat,Amendement AN,Amendement senat,N° article de la loi (AN),N° article de la loi (senat),Similarité
0,2058,COM-4,"APRÈS L'ARTICLE 14, insérer l'article suivan...",Après l'article 14 quater (nouveau) Insérer ...,ADDITIONNEL,14 QUATER (NOUVEAU),65%
1,2150,COM-7,Supprimer cet article.,Supprimer cet article.,12 QUATER,12 BIS A (NOUVEAU),100%
2,1949,COM-16,"APRÈS L'ARTICLE 15 BIS, insérer l'article su...",Après l'article 10 octies (nouveau) Insérer ...,ADDITIONNEL,10 OCTIES (NOUVEAU),85%
3,499,COM-18,"APRÈS L'ARTICLE 14, insérer l'article suivan...",Avant l'article 16 A (nouveau) Insérer un art...,ADDITIONNEL,16 A (NOUVEAU),11%
4,2317,COM-19,"APRÈS L'ARTICLE 11 DUOVICIES, insérer l'arti...",Après l'article 16 A (nouveau) Insérer un ar...,ADDITIONNEL,16 A (NOUVEAU),52%


In [ ]:
#%matplotlib inline
#df_amendements_matched["Simi"] = df_amendements_matched["Similarité"]\
#.apply(lambda x:int(x.replace("%", "")))
#df_amendements_matched.Simi.hist(bins=100)
#del df_amendements_matched["Simi"]

In [24]:
df_avis = pd.read_excel("tableau_sp.xls", skiprows=[0])[["N°", "Avis SAJ titre I"]].fillna("")

In [28]:
df_results_final = pd.merge(df_results, df_avis, left_on="N° AN", right_on="N°", how='left')
del df_results_final["N°"]
df_results_final.to_csv("amendements_communs_AN_senat.csv", index=False)
df_results_final.to_excel("amendements_communs_AN_senat.xls", index=False)
df_results_final.head()
print(len(df_results_final))

110
